<br>
<center>

# Assignments for Week 9

</center>

<div style="text-align: right; width: 15%; margin-left: auto;">
    
Jon Cannaday <br>
DSC 550 <br>
5/10/2024  

</div>


## Libraries

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import pandas as pd

## Import the dataset and ensure that it loaded properly.

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [4]:
path = r'C:\Users\Owner\OneDrive\Desktop\DataScience\2024_Spring_DataMining\Loan_Train.csv'
data = pd.read_csv(path)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB
None


## Prepare the data for modeling by performing the following steps:

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

   - #### Drop the column “Load_ID.”

In [5]:
data.drop(columns=['Loan_ID'], inplace=True)

   - #### Drop any rows with missing data.

In [6]:
data.dropna(inplace=True)

   - #### Convert the categorical features into dummy variables.

In [7]:
data = pd.get_dummies(data, drop_first=True)

In [9]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 480 entries, 1 to 613
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ApplicantIncome          480 non-null    int64  
 1   CoapplicantIncome        480 non-null    float64
 2   LoanAmount               480 non-null    float64
 3   Loan_Amount_Term         480 non-null    float64
 4   Credit_History           480 non-null    float64
 5   Gender_Male              480 non-null    bool   
 6   Married_Yes              480 non-null    bool   
 7   Dependents_1             480 non-null    bool   
 8   Dependents_2             480 non-null    bool   
 9   Dependents_3+            480 non-null    bool   
 10  Education_Not Graduate   480 non-null    bool   
 11  Self_Employed_Yes        480 non-null    bool   
 12  Property_Area_Semiurban  480 non-null    bool   
 13  Property_Area_Urban      480 non-null    bool   
 14  Loan_Status_Y            480 no

## Split the data into a training and test set, where the “Loan_Status” column is the target.

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [11]:
X = data.drop(columns=['Loan_Status_Y']) # Target
y = data['Loan_Status_Y'] #Features

# 80% : 20% , train : test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Create a pipeline with a min-max scaler and a KNN classifier (see section 15.3 in the Machine Learning with Python Cookbook).

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [24]:
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # MinMaxScaler
    ('knn', KNeighborsClassifier())  # KNN Classifier
])

## Fit a default KNN classifier to the data with this pipeline. Report the model accuracy on the test set. Note: Fitting a pipeline model works just like fitting a regular model.

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [25]:
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # MinMaxScaler
    ('knn', KNeighborsClassifier())  # KNN
])

# Fit Training data
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.78125


## Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10. (see section 15.3 in the Machine Learning with Python Cookbook).

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [26]:
param_grid = {
    'knn__n_neighbors': range(1, 11)  # 1 to 10
}

# Grid Search
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': range(1, 11)}, scoring='accuracy')

## Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [27]:
# Parameters and Score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'knn__n_neighbors': 3}
Best Score: 0.7423103212576898


## Find the accuracy of the grid search best model on the test set. Note: It is possible that this will not be an improvement over the default model, but likely it will be.

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [29]:
best_model = grid_search.best_estimator_

# Predict label
y_pred = best_model.predict(X_test)

# Accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Model Accuracy:", test_accuracy)

Model Accuracy: 0.7916666666666666


## Now, repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and random forest models with the hyperparameter values in section 12.3 of the Machine Learning with Python Cookbook.

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [ ]:
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classifier', None)  # Placeholder
])

# Search space
search_space = [
    # KNN
    {
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': list(range(1, 11))
    },
    # Logistic Regression
    {
        'classifier': [LogisticRegression()],
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'classifier__penalty': ['l1', 'l2']
    },
    # Random Forest
    {
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [10, 50, 100, 200],
        'classifier__max_depth': [None, 10, 20, 30]
    }
]

# Grid search
grid_search = GridSearchCV(pipeline, search_space, cv=5, scoring='accuracy')

# Fit Training data
grid_search.fit(X_train, y_train)

# Accuracy
best_model = grid_search.best_estimator_
best_accuracy = grid_search.best_score_

## What are the best model and hyperparameters found in the grid search? Find the accuracy of this model on the test set.

<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [33]:
print("Model:", best_model)
print("Accuracy:", best_accuracy)

Model: Pipeline(steps=[('scaler', MinMaxScaler()),
                ('classifier', RandomForestClassifier(max_depth=10))])
Accuracy: 0.810082023239918


## Summarize your results.
<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

The grid search determined the combination of MinMaxScaler & Random Forest Classifier with depth of 10 was ideal in this case.

This combination achieved an accuracy around 81.01% on the training data.